In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier




import gc #Trash Collection
import re #Regular Expression
import joblib #Export Pre-Processor/Model 

# import seaborn for more plotting options(built on top of matplotlib)
import seaborn as sns

In [ ]:

# import dataset application_train.csv an application_test.csv
df_train = pd.read_csv('application_train.csv')
df_test = pd.read_csv('application_test.csv')

#describe df_train and  df_test

df_train.describe()
df_test.describe()
print(df_train.shape)
print(df_test.shape)
#How the target labels are spread

sns.countplot(df_train.TARGET)
df_train['TARGET'].value_counts()

In [ ]:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Drop rows with NaN values
df_train = df_train.dropna()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('TARGET', axis=1), df_train['TARGET'], test_size=0.2)

# Create the random forest classifier
clf = RandomForestClassifier()

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Evaluate the classifier on the test data
accuracy = clf.score(X_test, y_test)


# accuracy
print('Accuracy: {:.2f}%'.format(accuracy * 100))

# roc_auc_score
print('ROC AUC Score: {:.2f}%'.format(roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]) * 100))

#confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, clf.predict(X_test))

#dataviz
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Create a confusion matrix
cm = confusion_matrix(y_test, clf.predict(X_test))

# Create a heatmap

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

# create confusion matrix.png 
plt.savefig('confusion_matrix.png')




In [ ]:
pip install mlflow
# mlflow
import mlflow
import mlflow.sklearn


# Create an experiment
mlflow.set_experiment('Home Credit Default Risk')

# Start a run
with mlflow.start_run():
        
        # Log the model
        mlflow.sklearn.log_model(clf, 'model')
        
        # Log the accuracy
        mlflow.log_metric('accuracy', accuracy)
        
        # Log the roc_auc_score
        mlflow.log_metric('roc_auc_score', roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))
        
        # Log the confusion matrix
        mlflow.log_artifact('confusion_matrix.png')
        

